# Install Libraries

In [1]:
!pip install selenium

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import requests
import pandas as pd
#from time import sleep
#import re

# Work

In [2]:
base_url = 'http://diariooficial.rn.gov.br/dei/dorn3/'

binary = r'C:\Program Files\Mozilla Firefox\firefox.exe'
options = Options()
options.headless = True
options.binary = binary

cap = DesiredCapabilities().FIREFOX
cap["marionette"] = True #optional

#driver = webdriver.Firefox(options=options, capabilities=cap, executable_path='drivers\geckodriver.exe')
driver = webdriver.Firefox(capabilities=cap, executable_path='drivers\geckodriver.exe')
driver.get(base_url)

In [52]:
materias = []

In [53]:
data = driver.find_element_by_class_name('ui-datepicker-days-cell-over.ui-datepicker-current-day')
ano = data.get_attribute('data-year')
mes = str(int(data.get_attribute('data-month')) + 1)
dia = data.find_element_by_tag_name('a').get_attribute('text')
data_formatada = (dia + '/0' + mes + '/' + ano if int(mes) < 10 else dia + '/' + mes + '/' + ano)
data_formatada = ('0' + data_formatada if int(dia) < 10 else data_formatada)
data_formatada

'10/10/2019'

In [7]:
def processar_linhas(link, text, blank):
    caminho = driver.find_element_by_id("lblSessaoPath").text
    lista_caminho = caminho.split("\\")[:-1]
    if blank != 0:
        [lista_caminho.append('') for b in range(blank)]
    if lista_caminho[0] == "DIÁRIO DA JUSTIÇA":
        lista_caminho.insert(0,'DIÁRIO OFICIAL')
    lista_caminho.insert(0, data_formatada)
    lista_caminho.append(text)
    lista_caminho.append(link)
    print(lista_caminho)
    return lista_caminho

In [8]:
def navegar(link, blank=5):
    link.click()
    global materias
    subsessoes = []
    try:
        subsessoes = driver.find_element_by_id("lblQtdMateriasPublicadas")
        if subsessoes.text == 'Nenhuma matéria encontrada nessa data para essa seção.':
            materia = processar_linhas(link=subsessoes.text, text='Não Possui', blank=blank)
            materias.append(materia)
            return []
        tabela = driver.find_element_by_id("dgDocumentos")
        files = tabela.find_elements_by_tag_name("a")
        for f in files:
            link = f.get_attribute("href")
            materia = processar_linhas(link=link, text=f.text, blank=blank)
            materias.append(materia)
        try:
            tabela = driver.find_element_by_id("dgSubSessoes")
            links = tabela.find_elements_by_tag_name("a")
            return links
        except:
            return []
    except:
        tabela = driver.find_element_by_id("dgSubSessoes")
        links = tabela.find_elements_by_tag_name("a")
    return links

#elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
#teste = navegar(elements)
#materias

In [9]:
def crawl_diario(link_sessao):
    #sleep(1)
    sub1 = navegar(link_sessao)
    if len(sub1) != 0:
        id1 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub1)+2)]
        for i1 in id1:
            #sleep(1)
            #print(i1)
            s1 = driver.find_element_by_id(i1)
            sub2 = navegar(s1, blank=4)
            if len(sub2) == 0:
                #sleep(1)
                driver.back()
            else:
                id2 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                       else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub2)+2)]
                for i2 in id2:
                    #sleep(1)
                    s2 = driver.find_element_by_id(i2)
                    sub3 = navegar(s2, blank=3)
                    if len(sub3) == 0:
                        #sleep(1)
                        driver.back()
                    else:
                        id3 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub3)+2)]
                        for i3 in id3:
                            #sleep(1)
                            s3 = driver.find_element_by_id(i3)
                            sub4 = navegar(s3, blank=2)
                            if len(sub4) == 0:
                                #sleep(1)
                                driver.back()
                            else:
                                id4 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                                       else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub4)+2)]
                                for i4 in id4:
                                    #sleep(1)
                                    s4 = driver.find_element_by_id(i4)
                                    sub5 = navegar(s4, blank=1)
                                    if len(sub5) == 0:
                                        #sleep(1)
                                        driver.back()
                                    else:
                                        id5 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                                               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub5)+2)]
                                        for i5 in id5:
                                            #sleep(1)
                                            s5 = driver.find_element_by_id(i5)
                                            sub6 = navegar(s5, blank=0)
                                            if len(sub6) == 0:
                                                #sleep(1)
                                                driver.back()
                                            if i5 == id5[-1]:
                                                #sleep(1)
                                                driver.back()
                                    if i4 == id4[-1]:
                                        #sleep(1)
                                        driver.back()
                            if i3 == id3[-1]:
                                #sleep(1)
                                driver.back()
                    if i2 == id2[-1]:
                        #sleep(1)
                        driver.back()
            if i1 == id1[-1]:
                #sleep(1)
                driver.back()

segunda versão

In [10]:
elements = ["Menu1_dgMenu_ctl0" + str(i) + "_LinkButton1" for i in range(2,10)]
#lista = navegar(sessoes[7])
#lista = []
for e in elements:
    s = driver.find_element_by_id(e)
    crawl_diario(s)
print(materias)
#lista

['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (ASSECOM)', 'DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI)', '', '', 'DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOTORISTAS 2019).doc', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20191010&id_doc=661668']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (ASSECOM)', 'DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI)', '', '', 'DEI - EXTRATO 1º TERMO DE ADITIVO (COBEL 2019)', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20191010&id_doc=661669']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (ASSECOM)', 'DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI)', 'DEI ASSEJUR', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICI

['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETÁRIA DE ESTADO DA ADMINISTRAÇÃO PENITENCIÁRIA (SEAP)', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DA AGRICULTURA, DA PECUÁRIA E DA PESCA ', 'CENTRAIS DE ABASTECIMENTO DO RN (CEASA)', 'PROGRAMA FARMACIA DE TODOS -CEASA/RN', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DA AGRICULTURA, DA PECUÁRIA E DA PESCA ', 'EMPRESA DE PESQUISA AGROPECUÁRIA DO RN (EMPARN)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DA AGRICULTURA, DA PECUÁRIA E DA PESCA ', 'INSTITUTO DE ASSISTÊNCIA TÉCNICA E EXTE

['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE SEGURANÇA PÚBLICA E DA DEFESA SOCIAL', '', '', '', 'Extrato Dispensa de Licitação nº 022-2019 - Proc. nº 00510028.002644-2018-48', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20191010&id_doc=661521']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE SEGURANÇA PÚBLICA E DA DEFESA SOCIAL', '', '', '', 'Extrato de Convênio - Extremoz - RN', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20191010&id_doc=661522']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE SEGURANÇA PÚBLICA E DA DEFESA SOCIAL', 'ACADEMIA DE POLICIA CIVIL -ACADEPOL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE SEGURANÇA PÚBLICA E DA DEFE

['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DO TRABALHO, DA HABITAÇÃO E DA ASSISTÊNCIA SOCIAL', '', '', '', '\\EDITAL DE BOLSISTA SETHAS-FAPERN – 2019 RESULTADO FINAL', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20191010&id_doc=661609']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DO TRABALHO, DA HABITAÇÃO E DA ASSISTÊNCIA SOCIAL', 'COMPANHIA ESTADUAL DE HABITAÇÃO E DESENVOLVIMENTO URBANO - CEHAB', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DO TRABALHO, DA HABITAÇÃO E DA ASSISTÊNCIA SOCIAL', 'FUNDAÇÃO DE ATENDIMENTO SOCIOEDUCATIVO DO ESTADO DO RIO GRANDE DO NORTE – FUNDASE/RN', '', '', 'TERMO DISPENSA LICITAÇÃO', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20191010&id_doc=661550']
['10/09/2019

['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DA INFRA-ESTRUTURA ', 'DEPARTAMENTO ESTADUAL DE TRANSITO (DETRAN) ', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DA JUSTIÇA E DA CIDADANIA ', 'CONSELHO ESTADUAL DE DIREITOS HUMANOS E CIDADANIA', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DA SEGURANÇA PÚBLICA E DA DEFESA SOCIAL ', 'ACADEMIA DE POLICIA CIVIL- ACADEPOL', '', '', 'Port. 081 Institui 1º CTEU', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20191010&id_doc=661648']
['10/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DA SEGURANÇA PÚBLICA E DA DEFESA SOCIAL ', 'ACADEMIA DE POLICIA CIVIL- ACADEPOL', '', '', 'Port 082 designa coor

...

['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VARELLA BARCA ', 'VARAS DE FAMÍLIA ', '4ª VARA DE FAMÍLIA ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VARELLA BARCA ', 'VARAS DE FAMÍLIA ', '5ª VARA DE FAMÍLIA ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VARELLA BARCA ', 'VARAS DE FAMÍLIA ', '6ª VARA DE FAMÍLIA ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'JUIZADOS ESPECIAIS CÍVEIS E CRIMINAIS', '1ª TURMA RECURSAL DOS JUIZADOS ESPECIAIS'

['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '3ª VARA DE FAMÍLIA ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '3ª VARA DE FAZENDA PÚBLICA ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '4ª VARA DE FAMÍLIA ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '4ª VARA DE FAZENDA PÚBLICA ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', '

['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CRIMINAIS ', '8ª VARA CRIMINAL ', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CRIMINAIS ', '9ª VARA CRIMINAL ', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'CORREGEDORIA DA JUSTIÇA ', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'TRIBUNAL DE JUSTIÇA ', 'ATAS E PAUTAS DE DISTRIBUIÇÃO E DE JULGAMENTO', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'TRIBUNAL DE JUSTIÇA ', 'CÂMARA CRIMINAL', '', '', 'Não Possui', 'Nenhuma matéri

['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '2ª VARA FAMILIA', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '3ª VARA CÍVEL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '3ª VARA CRIMINAL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '3ª VARA FAMÍLIA', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '4ª VARA CÍVEL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/

['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'ATOS DAS VARAS DO TRABALHO', 'VARA DO TRABALHO DE PAU DOS FERROS', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'ESTATÍSTICA', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'TABELA DE COEFICIENTES DE CORREÇÃO', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'TRIBUNAL PLENO E GABINETES', 'ACÓRDÃOS', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['10/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'TRIBUNAL PLENO E GABINETES', 'ATAS', '', ''

In [3]:
driver.quit()

In [12]:
df = pd.DataFrame(materias,columns=['data_publicacao', 'diario', 'secao', 'subsecao1', 'subsecao2', 'subsecao3', 'subsecao4', 'subsecao5', 'nome_materia', 'materia'])
df.head(3)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia
0,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),,,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
1,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),,,DEI - EXTRATO 1º TERMO DE ADITIVO (COBEL 2019),http://diariooficial.rn.gov.br/dei/dorn3/docvi...
2,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,,Não Possui,Nenhuma matéria encontrada nessa data para ess...
3,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSELHO DE DESENVOLVIMENTO DO ESTADO (CDE),,,,Não Possui,Nenhuma matéria encontrada nessa data para ess...
4,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSULTORIA GERAL DO ESTADO (CGE),,,,Não Possui,Nenhuma matéria encontrada nessa data para ess...


In [13]:
df[df['nome_materia'] == 'Não Possui'].diario.count()

306

In [14]:
df.shape

(461, 10)

In [16]:
filename = "diário - {}".format(data_formatada.replace('/','_'))
df.to_csv(('datasets/diários/' + filename))

# Pegar texto das matérias

In [18]:
materias_df = df[df['nome_materia'] != 'Não Possui'].copy()
materias_df.head(3)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia
0,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),,,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
1,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),,,DEI - EXTRATO 1º TERMO DE ADITIVO (COBEL 2019),http://diariooficial.rn.gov.br/dei/dorn3/docvi...
5,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONTROLADORIA GERAL DO ESTADO (CONTROL),,,,RESUMO 3º Termo Aditivo Prorrogação,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
10,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,AVISO RESULTADO DA LICITAÇÃO LICITAÇÃO TOMADA ...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
11,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,TERMO DE JUSTIFICATIVA Inversão de ordem crono...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


In [19]:
print(materias_df.materia.head(3))

0     http://diariooficial.rn.gov.br/dei/dorn3/docvi...
1     http://diariooficial.rn.gov.br/dei/dorn3/docvi...
5     http://diariooficial.rn.gov.br/dei/dorn3/docvi...
10    http://diariooficial.rn.gov.br/dei/dorn3/docvi...
11    http://diariooficial.rn.gov.br/dei/dorn3/docvi...
Name: materia, dtype: object


In [20]:
def pegar_frame(link):
    r = requests.get(link)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, "html.parser")
    frame = soup.find_all('frame')[1]
    frame_link = frame.get('src')
    url_materia_oficial = base_url + frame_link
    return url_materia_oficial

In [21]:
materias_df['link_materia_oficial'] = materias_df['materia'].apply(pegar_frame)
materias_df.head(3)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia,link_materia_oficial
0,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),,,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
1,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),,,DEI - EXTRATO 1º TERMO DE ADITIVO (COBEL 2019),http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
5,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONTROLADORIA GERAL DO ESTADO (CONTROL),,,,RESUMO 3º Termo Aditivo Prorrogação,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
10,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,AVISO RESULTADO DA LICITAÇÃO LICITAÇÃO TOMADA ...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
11,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,TERMO DE JUSTIFICATIVA Inversão de ordem crono...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...


In [22]:
print(materias_df.link_materia_oficial.iloc[50])

http://diariooficial.rn.gov.br/dei/dorn3/documentos/00000001/20191010/661601.htm


In [23]:
def pegar_texto(link):
    global cont
    r = requests.get(link)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, "html.parser")
    #if len(soup.find_all('table')) > 0:
    #    soup('table')[0].decompose()
    conteudo_materia = soup.find_all('body')[0].text.strip()
    #conteudo_materia = re.sub("(<!--.*?-->)", "", noticia, flags=re.DOTALL)
    return conteudo_materia

In [24]:
materias_df['texto'] = materias_df['link_materia_oficial'].apply(pegar_texto)
materias_df.head(3)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia,link_materia_oficial,texto
0,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),,,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,DEPARTAMENTO ESTADUAL DE IMPRENSA\nEXTRATO\r\n...
1,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),,,DEI - EXTRATO 1º TERMO DE ADITIVO (COBEL 2019),http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,DEPARTAMENTO ESTADUAL DE IMPRENSA\nEXTRATO\r\n...
5,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONTROLADORIA GERAL DO ESTADO (CONTROL),,,,RESUMO 3º Termo Aditivo Prorrogação,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,RESUMO\r\nDE TERMO ADITIVO\nProcesso\r\nSEI! n...
10,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,AVISO RESULTADO DA LICITAÇÃO LICITAÇÃO TOMADA ...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,AVISO\nRESULTADO\r\nDA LICITAÇÃO\nLICITAÇÃO TO...
11,10/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,TERMO DE JUSTIFICATIVA Inversão de ordem crono...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,TERMO\r\nDE JUSTIFICATIVA\nInversão\r\nde orde...


In [25]:
print(materias_df.texto.iloc[50])

PGE/RN
SECRETARIA GERAL
RELAÇÃO DE DISTRIBUIÇÃO POR PROCURADOR NO
PERÍODO DE 09/10/2019 ATÉ 09/10/2019
CITAÇÕES, INTIMAÇÕES E NOTIFICAÇÕES






Setor:
CONTENCIOSO





Procurador


Processo





ÁLVARO VERAS CASTRO
  MELO


[0833797-41.2017.8.20.5001] [0852654-72.2016.8.20.5001]
[0825099-75.2019.8.20.5001] [0849596-90.2018.8.20.5001]
[0805183-31.2014.8.20.5001] [0801709-07.2019.8.20.5121]
[0828693-68.2017.8.20.5001] [0833643-91.2015.8.20.5001]
[0830601-92.2019.8.20.5001] Total de Processos (9)




JULIANA DE MORAIS GUERRA


[00410002.003131/2019-89] [0804970-49.2019.8.20.5001]
[0800203-65.2019.8.20.5001] [0800983-48.2011.8.20.0001]
[0804087-78.2014.8.20.5001] [0843670-02.2016.8.20.5001]
[0817262-03.2018.8.20.5001] [2016.000446-1] [0800616-85.2019.8.20.5128]
Total de Processos (9)




LUCAS CHRISTOVAM DE OLIVEIRA


[0850145-03.2018.8.20.5001] [0827835-66.2019.8.20.5001]
[0855797-35.2017.8.20.5001] [0854065-53.2016.8.20.5001]
[0834703-31.2017.8.20.5001] [0826586-80.2019.8.20.5001]
[0842

In [27]:
filename = "matérias_diário - {}".format(data_formatada.replace('/','_'))
materias_df.to_csv(('datasets/matérias/' + filename), index=False)

# Lendo datasets mais antigos

In [46]:
#diario = pd.read_csv('datasets\diários\diário - 28_09_2019')
diario = pd.read_csv('datasets\diários\diário - 01_10_2019')
diario.head(3)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia
0,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,NaN,Não Possui,Nenhuma matéria encontrada nessa data para ess...
1,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSELHO DE DESENVOLVIMENTO DO ESTADO (CDE),NaN,NaN,NaN,Não Possui,Nenhuma matéria encontrada nessa data para ess...
2,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSULTORIA GERAL DO ESTADO (CGE),NaN,NaN,NaN,Não Possui,Nenhuma matéria encontrada nessa data para ess...


In [47]:
materias_diario = diario[diario['nome_materia'] != 'Não Possui'].copy()
materias_diario.head(3)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia
7,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
8,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,NaN,NaN,NaN,SIN - ex-CV-018-2019-DETRAN-DER.doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
10,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,DEPARTAMENTO ESTADUAL DE TRÂNSITO (DETRAN),NaN,NaN,Extrato de Contrato,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


In [48]:
materias_diario['link_materia_oficial'] = materias_diario['materia'].apply(pegar_frame)
materias_diario.head(3)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia,link_materia_oficial
7,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
8,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,NaN,NaN,NaN,SIN - ex-CV-018-2019-DETRAN-DER.doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
10,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,DEPARTAMENTO ESTADUAL DE TRÂNSITO (DETRAN),NaN,NaN,Extrato de Contrato,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...


In [49]:
materias_diario['texto'] = materias_diario['link_materia_oficial'].apply(pegar_texto)
materias_diario.head(3)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia,link_materia_oficial,texto
7,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO\r\nDE DISPENSA DE LICITAÇÃO\nProcesso...
8,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,NaN,NaN,NaN,SIN - ex-CV-018-2019-DETRAN-DER.doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,*\r\nEXTRATO DO CONVÊNIO N° 018/2019 - SIN\n \...
10,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,DEPARTAMENTO ESTADUAL DE TRÂNSITO (DETRAN),NaN,NaN,Extrato de Contrato,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,Secretaria de InfraEstrutura.\r\nDepartamento ...


In [50]:
filename = "matérias_diário - {}".format(materias_diario.iloc[0,0].replace('/','_'))
materias_diario.to_csv(('datasets/matérias/' + filename), index=False)

# O que fazer
##### Seleção de data
    Iterar por data (opcional)
##### Criar Funções (fazendo)
##### Minimizar gambiarras
##### Criar arquivo .py

# Corrigindo datas

#### Diário

In [54]:
file_diario = pd.read_csv('datasets\diários\diário - 10_10_2019') #.iloc[:,1:]
file_diario.head(1)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia
0,10/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),NaN,NaN,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


In [52]:
file_diario.data_publicacao = '10/10/2019'
file_diario.to_csv('datasets\diários\diário - 10_10_2019', index=False)

#### Matérias

In [10]:
file_materia = pd.read_csv('datasets\matérias\matérias_diário - 01_10_2019') #.iloc[:,1:]
file_materia.head(1)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia,link_materia_oficial,texto
0,01/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO\r\nDE DISPENSA DE LICITAÇÃO\nProcesso...


In [8]:
file_materia.data_publicacao = '01/10/2019'
file_materia.to_csv('datasets\matérias\matérias_diário - 01_10_2019', index=False)

# Corrigindo nome das colunas

In [1]:
import glob

In [8]:
diarios = glob.glob('datasets\diários\diário - *')
materias = glob.glob('datasets\matérias\matérias_diário - *')
print(diarios, materias)

['datasets\\diários\\diário - 01_10_2019', 'datasets\\diários\\diário - 03_10_2019', 'datasets\\diários\\diário - 08_10_2019', 'datasets\\diários\\diário - 09_10_2019', 'datasets\\diários\\diário - 10_10_2019', 'datasets\\diários\\diário - 28_09_2019'] ['datasets\\matérias\\matérias_diário - 01_10_2019', 'datasets\\matérias\\matérias_diário - 03_10_2019', 'datasets\\matérias\\matérias_diário - 08_10_2019', 'datasets\\matérias\\matérias_diário - 09_10_2019', 'datasets\\matérias\\matérias_diário - 10_10_2019', 'datasets\\matérias\\matérias_diário - 28_09_2019']


#### Diários

In [12]:
new_cols = ['secao','subsecao1','subsecao2','subsecao3','subsecao4','subsecao5']

for d in diarios:
    diario = pd.read_csv(d)
    diario.rename(columns=dict(zip(diario.columns[2:8], new_cols)),inplace=True)
    diario.to_csv(d, index=False)
    display(diario.head(1))

,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,01/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,NaN,Não Possui,Nenhuma matéria encontrada nessa data para ess...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,03/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,NaN,Não Possui,Nenhuma matéria encontrada nessa data para ess...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,08/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,NaN,Não Possui,Nenhuma matéria encontrada nessa data para ess...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,09/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),NaN,NaN,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,10/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),NaN,NaN,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,28/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),NaN,NaN,DEI - EXTRATO DISPENSA DE LICITAÇÃO (TECPEL).doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


#### Matérias

In [13]:
for m in materias:
    materia = pd.read_csv(m)
    materia.rename(columns=dict(zip(materia.columns[2:8], new_cols)),inplace=True)
    materia.to_csv(m, index=False)
    display(materia.head(1))

,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
0,01/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO\r\nDE DISPENSA DE LICITAÇÃO\nProcesso...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
0,03/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO DO PRIMEIRO TERMO ADITIVO AO TERMO DE ...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
0,08/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),NaN,NaN,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,DEPARTAMENTO ESTADUAL DE IMPRENSA\nEXTRATO\r\n...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
0,09/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO DO II TERMO ADITIVO AO CONTRATO\r\r\nD...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
0,10/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),NaN,NaN,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,DEPARTAMENTO ESTADUAL DE IMPRENSA\nEXTRATO\r\n...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
0,28/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO\r\nDE DISPENSA DE LICITAÇÃO\nProcesso...


# Teste